In [ ]:
pip install --upgrade pip

In [2]:
import warnings, os
warnings.filterwarnings("ignore")
from dotenv import load_dotenv
load_dotenv('env')
import pandas as pd
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en.stop_words import STOP_WORDS as stopwords
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
import spacy
from importlib import reload
from modules import DataCollection, Support
reload(Support)
reload(DataCollection)
from modules.Support import Corpus, Business, TokenCleaner

/opt/conda/lib/python3.7/site-packages/ipykernel/pylab/backend_inline.py:164: DeprecationWarning: `configure_inline_support` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.configure_inline_support()`
  configure_inline_support(ip, backend)
/opt/conda/lib/python3.7/site-packages/botocore/httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
  from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
/opt/conda/lib/python3.7/site-packages/scipy/io/matlab/mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.or

In [3]:
import ray, os
ray.init(num_cpus=96)
os.environ["MODIN_ENGINE"] = "ray"
import modin.pandas as md

/opt/conda/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2022-11-16 05:56:34,654	INFO worker.py:1528 -- Started a local Ray instance.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdoc

In [25]:
s3_client = DataCollection.s3_client()

if not os.path.exists("restaurants"):
    os.mkdir("restaurants")
    
for file_name in ['review.csv', 'business.csv']:
    if not os.path.exists("restaurants/%s" % file_name):
        s3_client.download_file('s3fld','599Team12/restaurants/%s'% file_name,'restaurants/%s'% file_name) 
    
s3_client.download_file('s3fld','599Team12/negative-words.txt','negative-words.txt') 

In [28]:
with open('negative-words.txt','r') as f:
    negative_words = f.read().split('\n')
    
# remove heafer information
negative_words = negative_words[35:]
pd.DataFrame(negative_words).sample(5)

,0
3326,pokey
1789,frustrates
4483,unlicensed
904,degenerately
4396,uncooperative


In [4]:
business = md.read_csv('restaurants/business.csv', compression = 'zip')
review = md.read_csv('restaurants/review.csv', compression = 'zip')

2022-11-16 05:56:40,502	WARNING __init__.py:192 -- DeprecationWarning: `ray.worker.global_worker` is a private attribute and access will be removed in a future Ray version.


In [5]:
negative_reviews = review[review['sentiment'] == 'negative']
negative_reviews = negative_reviews.reset_index(drop=True)
negative_reviews = negative_reviews[['review_id', 'text', 'date', 'sentiment']]

In [6]:
tc = TokenCleaner(return_as_string=False)
negative_reviews['text_cleaned'] = negative_reviews['text'].apply(tc.CleanText)
negative_reviews.head()

,review_id,text,date,sentiment,text_cleaned
0,JrIxlS1TzJ-iCu79ul40cQ,I am a long term frequent customer of this est...,2015-09-23 23:10:31,negative,"[long, term, frequent, custom, thi, establish,..."
1,elqRpX9T3YwL07uLNtN3Bg,I at least have to give this restaurant two st...,2015-02-02 04:29:13,negative,"[least, give, thi, restaur, two, star, due, de..."
2,RB8UpF_kT2xoOC51OzXEeA,"Straight to the point, it's cheap, it tastes a...",2017-07-08 18:58:42,negative,"[straight, point, cheap, tast, feel, cheap, go..."
3,TcCcHzc3L6Aboq3DteEfZA,If you want to pay for everything a la carte t...,2014-08-24 20:14:12,negative,"[want, pay, everyth, la, cart, thi, place, foo..."
4,yCbzm_d4cHpfUX00Ljvgmg,"We started with the artichoke dip, big mistake...",2017-09-08 23:20:19,negative,"[start, artichok, dip, big, mistak, wa, cold, ..."


In [7]:
# use itertools to count  all the words in the negative_reviews.text_cleaned column
word_counts = Counter(chain.from_iterable(negative_reviews['text_cleaned']))

# convert the word_counts dict to a dataframe and sort it by the count
word_counts = pd.DataFrame.from_dict(word_counts, orient='index').reset_index()
word_counts.columns = ['word', 'count']
word_counts = word_counts.sort_values(by='count', ascending=False)
word_counts = word_counts.loc[word_counts['word'].isin(negative_words)]
word_counts = word_counts.reset_index(drop=True)
word_counts

,word,count
0,disappoint,164531
1,bad,162671
2,cold,106007
3,rude,95870
4,worst,92165
...,...,...
1283,strenuous,1
1284,discomfit,1
1285,arduous,1
1286,wasteful,1


In [31]:
for index, row in word_counts.iterrows():
    # count how many time each root word was used in review with negative sentiment
    word_to_look_for = row.word
    occurance = negative_reviews['text_cleaned'].apply(lambda x: x.count(word_to_look_for)).sum()
    word_counts.loc[index, 'negative_use'] = occurance
    print('{:0>3} | "{}" was used {:,} times in the negative reviews.'.format(index, word_to_look_for, occurance))

000 | "scare" was used 1,928 times in the negative reviews.
001 | "scare" was used 1,928 times in the negative reviews.
002 | "scare" was used 1,928 times in the negative reviews.
003 | "scare" was used 1,928 times in the negative reviews.
004 | "scare" was used 1,928 times in the negative reviews.
005 | "scare" was used 1,928 times in the negative reviews.
006 | "scare" was used 1,928 times in the negative reviews.
007 | "scare" was used 1,928 times in the negative reviews.
008 | "scare" was used 1,928 times in the negative reviews.
009 | "scare" was used 1,928 times in the negative reviews.
010 | "scare" was used 1,928 times in the negative reviews.
011 | "scare" was used 1,928 times in the negative reviews.
012 | "scare" was used 1,928 times in the negative reviews.
013 | "scare" was used 1,928 times in the negative reviews.
014 | "scare" was used 1,928 times in the negative reviews.
015 | "scare" was used 1,928 times in the negative reviews.
016 | "scare" was used 1,928 times in th

In [33]:
s3_client = DataCollection.s3_client()

if not os.path.exists("analysis"):
    os.mkdir("analysis")
    
negative_reviews.to_csv('analysis/negative_reviews.csv', compression = 'zip')
s3_client.upload_file('analysis/negative_reviews.csv', 's3fld', '599Team12/analysis/negative_reviews.csv', ExtraArgs={'ACL': 'public-read'})

word_counts.to_csv('analysis/word_counts.csv', compression = 'zip')
s3_client.upload_file('analysis/word_counts.csv', 's3fld', '599Team12/analysis/word_counts.csv', ExtraArgs={'ACL': 'public-read'})